In [1]:
from utils.initialization import initialize
import pandas as pd
from dataset.dataset import SaladsDataset
from utils.pm_utils import discover_dk_process, remove_duplicates_dataset, remove_duplicates_trace, conformance_measure
from denoisers.ConditionalUnetMatrixDenoiser import ConditionalUnetMatrixDenoiser
from utils.Config import Config
import json
import pickle as pkl
import torch

In [2]:
with open("config.json", "r") as f:
    cfg_json = json.load(f)
    cfg = Config(**cfg_json)
with open(cfg.data_path, "rb") as f:
    dataset = pkl.load(f)

In [3]:
salads_dataset = SaladsDataset(dataset['target'], dataset['stochastic'])

In [4]:
dk_process_model, dk_init_marking, dk_final_marking = discover_dk_process(salads_dataset, cfg,
                                                                          preprocess=remove_duplicates_dataset)

In [5]:
test_trace_one_hot = salads_dataset[0:10][0].permute(0, 2, 1)
test_traces = torch.argmax(test_trace_one_hot, dim=1).to('cpu')
test_traces

tensor([[17, 17, 17,  ..., 19, 19, 19],
        [17, 17, 17,  ..., 19, 19, 19],
        [17, 17, 17,  ..., 19, 19, 19],
        ...,
        [17, 17, 17,  ..., 19, 19, 19],
        [17, 17, 17,  ..., 18, 18, 18],
        [17, 17, 17,  ..., 18, 18, 18]])

In [6]:
traces = [remove_duplicates_trace(test_trace).tolist() for test_trace in test_traces]
traces

[[17, 11, 13, 4, 5, 2, 3, 0, 1, 15, 8, 7, 9, 6, 10, 16, 14, 18, 19],
 [17, 11, 13, 4, 5, 2, 3, 0, 1, 15, 8, 7, 9, 6, 10, 16, 14, 18, 19],
 [17, 11, 13, 4, 5, 2, 3, 0, 1, 15, 8, 7, 9, 6, 10, 16, 14, 18, 19],
 [17, 11, 13, 4, 5, 2, 3, 0, 1, 15, 8, 7, 9, 6, 10, 16, 14, 18, 19],
 [17,
  2,
  3,
  0,
  1,
  0,
  1,
  15,
  1,
  4,
  5,
  4,
  5,
  11,
  12,
  13,
  12,
  13,
  7,
  8,
  6,
  9,
  15,
  16,
  18,
  19],
 [17,
  2,
  3,
  0,
  1,
  0,
  1,
  15,
  1,
  4,
  5,
  4,
  5,
  11,
  12,
  13,
  12,
  13,
  7,
  8,
  6,
  9,
  15,
  16,
  18,
  19],
 [17,
  2,
  3,
  0,
  1,
  0,
  1,
  15,
  1,
  4,
  5,
  4,
  5,
  11,
  12,
  13,
  12,
  13,
  7,
  8,
  6,
  9,
  15,
  16,
  18,
  19],
 [17,
  2,
  3,
  0,
  1,
  0,
  1,
  15,
  1,
  4,
  5,
  4,
  5,
  11,
  12,
  13,
  12,
  13,
  7,
  8,
  6,
  9,
  15,
  16,
  18,
  19],
 [17,
  7,
  8,
  9,
  6,
  10,
  11,
  12,
  11,
  12,
  13,
  2,
  3,
  0,
  1,
  4,
  5,
  4,
  5,
  15,
  16,
  14,
  18],
 [17,
  7,
  8,
  9,
  6,
  1

In [7]:
conformance_measure(test_traces, dk_process_model, dk_init_marking, dk_final_marking, cfg.activity_names, limit=None, approximate=False)

aligning log, completed variants ::   0%|          | 0/3 [00:00<?, ?it/s]

[{'alignment': [('action_start', 'action_start'),
   ('>>', None),
   ('peel_cucumber', 'peel_cucumber'),
   ('>>', None),
   ('>>', None),
   ('>>', None),
   ('place_cucumber_into_bowl', 'place_cucumber_into_bowl'),
   ('>>', None),
   ('>>', None),
   ('cut_lettuce', 'cut_lettuce'),
   ('place_lettuce_into_bowl', 'place_lettuce_into_bowl'),
   ('>>', None),
   ('cut_cheese', 'cut_cheese'),
   ('>>', None),
   ('>>', None),
   ('place_cheese_into_bowl', 'place_cheese_into_bowl'),
   ('>>', None),
   ('>>', None),
   ('cut_tomato', 'cut_tomato'),
   ('>>', None),
   ('>>', None),
   ('>>', None),
   ('>>', None),
   ('place_tomato_into_bowl', 'place_tomato_into_bowl'),
   ('>>', None),
   ('mix_ingredients', 'mix_ingredients'),
   ('>>', None),
   ('>>', None),
   ('>>', None),
   ('add_oil', 'add_oil'),
   ('add_vinegar', 'add_vinegar'),
   ('add_pepper', 'add_pepper'),
   ('>>', None),
   ('>>', None),
   ('add_salt', 'add_salt'),
   ('>>', None),
   ('>>', None),
   ('>>', None),
 

In [10]:
from timeit import timeit

print(timeit(lambda: conformance_measure([t[:50] for t in test_traces], dk_process_model, dk_init_marking, dk_final_marking, cfg.activity_names), number=1))
print(timeit(lambda: conformance_measure([t[:100] for t in test_traces], dk_process_model, dk_init_marking, dk_final_marking, cfg.activity_names), number=1))
print(timeit(lambda: conformance_measure([t[:500] for t in test_traces], dk_process_model, dk_init_marking, dk_final_marking, cfg.activity_names), number=1))
print(timeit(lambda: conformance_measure([t[:800] for t in test_traces], dk_process_model, dk_init_marking, dk_final_marking, cfg.activity_names), number=1))
print(timeit(lambda: conformance_measure([t[:1000] for t in test_traces], dk_process_model, dk_init_marking, dk_final_marking, cfg.activity_names), number=1))
print(timeit(lambda: conformance_measure([t[:1500] for t in test_traces], dk_process_model, dk_init_marking, dk_final_marking, cfg.activity_names), number=1))
print(timeit(lambda: conformance_measure([t[:8000] for t in test_traces], dk_process_model, dk_init_marking, dk_final_marking, cfg.activity_names), number=1))

0.01122919999761507
0.012155300006270409


aligning log, completed variants ::   0%|          | 0/3 [00:00<?, ?it/s]

0.03932599996915087


aligning log, completed variants ::   0%|          | 0/3 [00:00<?, ?it/s]

0.04999829997541383


...

0.0599794999579899


aligning log, completed variants ::   0%|          | 0/3 [00:00<?, ?it/s]

0.09965079999528825


aligning log, completed variants ::   0%|          | 0/3 [00:00<?, ?it/s]

0.43868630001088604
